In [ ]:
### CNN of choice

# InceptionV3

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/COGS181_final/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/COGS181_final


In [ ]:
# this cell is just for intial downloading of the img files
'''
import kagglehub
import shutil
import os

# AI Human Faces
ai_kaggle_path = kagglehub.dataset_download("almightyj/person-face-dataset-thispersondoesnotexist")
ai_drive_path = '/content/drive/MyDrive/COGS181_final/aifaces'
shutil.move(ai_kaggle_path,ai_drive_path)

# real human faces
human_original_path = "/content/drive/MyDrive/COGS181_final/images1024x1024"
human_new_path = "/content/drive/MyDrive/COGS181_final/humanfaces"

all_images = []
for root, _, files in os.walk(human_original_path):
    for file in files:
        if file.lower().endswith((".png", ".jpg", ".jpeg")):  # Ensure valid image types
            all_images.append(os.path.join(root, file))  # Store full image paths

100%|██████████| 4.45G/4.45G [00:57<00:00, 83.3MB/s]

Extracting files...


Total images found: 67000


In [ ]:
# down samples ffhq dataset of 70000 human faces images to 10000 and saves in drive
'''
import random

down_sampled_images = random.sample(all_images, 10000)

# add to down sampled folder in drive
for img_path in down_sampled_images:
  img_name = os.path.basename(img_path)
  destination = os.path.join(human_new_path, img_name)
  shutil.copy(img_path, destination)
'''

Total images in humanfaces: 10000


In [4]:
#check sizes of ai and real image files
import os

ai_folder_path = "/content/drive/MyDrive/COGS181_final/training_data/aifaces"
ai_files = os.listdir(ai_folder_path)
ai_image_files = [f for f in ai_files if f.lower().endswith((".png", ".jpg", ".jpeg"))]

real_folder_path = "/content/drive/MyDrive/COGS181_final/training_data/humanfaces"
real_files = os.listdir(real_folder_path)
real_image_files = [f for f in real_files if f.lower().endswith((".png", ".jpg", ".jpeg"))]

print(f"Total real face images {len(ai_image_files)}")
print(f"Total ai face images {len(real_image_files)}")

Total real face images 10000
Total ai face images 10000


In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
# make training directory
train_dir = "/content/drive/MyDrive/COGS181_final/training_data"

In [3]:
# data generator

# ImageDataGenerator has lots of parameters to diversify the image variations
train_data_gen = ImageDataGenerator(
    rescale = 1./255,
    # these are all randomizers to improve generalizability
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',

    validation_split = .2
)

In [4]:
batch = 32
img_size = (299,299)

# training data generator
train_gen = train_data_gen.flow_from_directory(
    train_dir,
    target_size = img_size,
    batch_size = batch,
    class_mode = 'binary',
    subset = 'training'
)

Found 16000 images belonging to 2 classes.


In [5]:
# validation data generator
val_gen = train_data_gen.flow_from_directory(
    train_dir,
    target_size = img_size,
    batch_size = batch,
    class_mode = 'binary',
    subset = 'validation'
)

Found 4000 images belonging to 2 classes.


In [6]:
# load inception v3
base_model = InceptionV3(weights = 'imagenet', include_top = False, input_shape = (299,299,3))
base_model.trainable = False

# customize some layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu", kernel_regularizer=l2(0.01))(x) # alt method for fcl
x = Dropout(0.4)(x) # regularize
x = Dense(1, activation="sigmoid")(x)  # Binary classification (0 = real, 1 = AI)

# final model structure (base inceptionv3 + custom layers for better image processing)
model = Model(inputs = base_model.input, outputs = x)


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 299, 299, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 149, 149, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 149, 149, 32)   │             96 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 149, 149, 32)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 147, 147, 32)   │          9,216 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 147, 147, 32)   │             96 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 147, 147, 32)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 147, 147, 64)   │         18,432 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 147, 147, 64)   │            192 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 147, 147, 64)   │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 73, 73, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 73, 73, 80)     │          5,120 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 73, 73, 80)     │            240 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 73, 73, 80)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 71, 71, 192)    │        138,240 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 71, 71, 192)    │            576 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activat

 Total params: 22,327,585 (85.17 MB)

 Trainable params: 524,801 (2.00 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# where the trained model will save to incrimentally
checkpoint_path = "/content/drive/MyDrive/COGS181_final/inception_v3_best_model_v2.h5"

# Create a checkpoint callback
checkpoint_callback = ModelCheckpoint(
    checkpoint_path,        # Filepath to save model
    monitor = "val_accuracy", # Save best model based on validation accuracy
    save_best_only = True,
    save_weights_only = False,
    verbose = 1
)

# Reduce learning rate when validation loss plateaus
reduce_lr = ReduceLROnPlateau(
    monitor = "val_loss",
    factor = 0.5,
    patience = 2,
    min_lr=1e-6
)

# Stop training if validation accuracy stops improving for 5 epochs
early_stop = EarlyStopping(
    monitor = "val_accuracy",
    patience = 5,
    restore_best_weights = True,
    verbose = 1
)

# Train the model with the callbacks
trained_model = model.fit(
    train_gen,
    validation_data = val_gen,
    epochs = 10,
    steps_per_epoch = 100,
    validation_steps = 50,
    callbacks = [checkpoint_callback, reduce_lr, early_stop],
    verbose = 1
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 27s/step - accuracy: 0.6045 - loss: 4.9403 
Epoch 1: val_accuracy improved from -inf to 0.70438, saving model to /content/drive/MyDrive/COGS181_final/inception_v3_best_model_v2.h5


100/100 ━━━━━━━━━━━━━━━━━━━━ 4091s 41s/step - accuracy: 0.6049 - loss: 4.9373 - val_accuracy: 0.7044 - val_loss: 4.0952 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 22s/step - accuracy: 0.7007 - loss: 3.8947 
Epoch 2: val_accuracy improved from 0.70438 to 0.73438, saving model to /content/drive/MyDrive/COGS181_final/inception_v3_best_model_v2.h5


100/100 ━━━━━━━━━━━━━━━━━━━━ 3162s 32s/step - accuracy: 0.7008 - loss: 3.8928 - val_accuracy: 0.7344 - val_loss: 3.3090 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 18s/step - accuracy: 0.7115 - loss: 3.1860 
Epoch 3: val_accuracy did not improve from 0.73438
100/100 ━━━━━━━━━━━━━━━━━━━━ 2536s 25s/step - accuracy: 0.7116 - loss: 3.1846 - val_accuracy: 0.7250 - val_loss: 2.7642 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 16s/step - accuracy: 0.7184 - loss: 2.6816 
Epoch 4: val_accuracy improved from 0.73438 to 0.73813, saving model to /content/drive/MyDrive/COGS181_final/inception_v3_best_model_v2.h5


100/100 ━━━━━━━━━━━━━━━━━━━━ 2173s 22s/step - accuracy: 0.7184 - loss: 2.6806 - val_accuracy: 0.7381 - val_loss: 2.3638 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.7406 - loss: 2.2915 
Epoch 5: val_accuracy improved from 0.73813 to 0.74563, saving model to /content/drive/MyDrive/COGS181_final/inception_v3_best_model_v2.h5


100/100 ━━━━━━━━━━━━━━━━━━━━ 1758s 18s/step - accuracy: 0.7406 - loss: 2.2908 - val_accuracy: 0.7456 - val_loss: 2.0559 - learning_rate: 1.0000e-04
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.7364 - loss: 2.0130 
Epoch 6: val_accuracy improved from 0.74563 to 0.75375, saving model to /content/drive/MyDrive/COGS181_final/inception_v3_best_model_v2.h5


100/100 ━━━━━━━━━━━━━━━━━━━━ 1876s 19s/step - accuracy: 0.7363 - loss: 2.0125 - val_accuracy: 0.7538 - val_loss: 1.8256 - learning_rate: 1.0000e-04
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.7367 - loss: 1.7959 
Epoch 7: val_accuracy improved from 0.75375 to 0.75750, saving model to /content/drive/MyDrive/COGS181_final/inception_v3_best_model_v2.h5


100/100 ━━━━━━━━━━━━━━━━━━━━ 1785s 18s/step - accuracy: 0.7367 - loss: 1.7955 - val_accuracy: 0.7575 - val_loss: 1.6368 - learning_rate: 1.0000e-04
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.7529 - loss: 1.6143 
Epoch 8: val_accuracy improved from 0.75750 to 0.77062, saving model to /content/drive/MyDrive/COGS181_final/inception_v3_best_model_v2.h5


100/100 ━━━━━━━━━━━━━━━━━━━━ 1819s 18s/step - accuracy: 0.7529 - loss: 1.6139 - val_accuracy: 0.7706 - val_loss: 1.4838 - learning_rate: 1.0000e-04
Epoch 9/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.7430 - loss: 1.4735 
Epoch 9: val_accuracy improved from 0.77062 to 0.78312, saving model to /content/drive/MyDrive/COGS181_final/inception_v3_best_model_v2.h5


100/100 ━━━━━━━━━━━━━━━━━━━━ 1740s 17s/step - accuracy: 0.7431 - loss: 1.4732 - val_accuracy: 0.7831 - val_loss: 1.3529 - learning_rate: 1.0000e-04
Epoch 10/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.7473 - loss: 1.3575 
Epoch 10: val_accuracy did not improve from 0.78312
100/100 ━━━━━━━━━━━━━━━━━━━━ 1669s 17s/step - accuracy: 0.7474 - loss: 1.3573 - val_accuracy: 0.7538 - val_loss: 1.2670 - learning_rate: 1.0000e-04
Restoring model weights from the end of the best epoch: 9.


In [10]:
# testing fully trained model

from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

model_path = "/content/drive/MyDrive/COGS181_final/inception_v3_best_model_v2.h5"
face_trained_model = load_model(model_path)

def realorfake(img_path, model):
  img = image.load_img(img_path, target_size=(299, 299))
  img_array = image.img_to_array(img) / 255.0
  img_array = np.expand_dims(img_array, axis=0)

  model_guess = model.predict(img_array)[0][0]
  if model_guess > .5:
    print('Prediction: This face is AI generated')
  else:
    print('Prediction: This face is real')

  print(f'Prediction Probability: {float(model_guess):.4f}')

  return model_guess


In [12]:
test_img = "/content/drive/MyDrive/COGS181_final/test_images/real/IMG_9933.png"
realorfake(test_img, face_trained_model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 793ms/step
Prediction: This face is AI generated
Prediction Probability: 0.9406


np.float32(0.9406469)

In [ ]:
face_trained_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 299, 299, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 149, 149, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 149, 149, 32)   │             96 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 149, 149, 32)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 147, 147, 32)   │          9,216 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 147, 147, 32)   │             96 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 147, 147, 32)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 147, 147, 64)   │         18,432 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 147, 147, 64)   │            192 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 147, 147, 64)   │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 73, 73, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 73, 73, 80)     │          5,120 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 73, 73, 80)     │            240 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 73, 73, 80)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 71, 71, 192)    │        138,240 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 71, 71, 192)    │            576 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activat

 Total params: 22,327,587 (85.17 MB)

 Trainable params: 524,801 (2.00 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

 Optimizer params: 2 (12.00 B)